In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
import numpy as np

c:\Users\chacon1\AppData\Local\Programs\Python\Python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
FSIS_DATA = "../data/fsis-processors-with-location.csv"
SIC_CODE = "2015" # Poultry Slaughtering and Processing
NAICS_CODE = "311615" # Poultry Processing
INFOGROUP_2022 = ["../data/2022_Business_Academic_QCQ.txt"]
# df = pd.read_csv(INFOGROUP_2022[0], nrows=10000, dtype=str)


In [3]:
# df = pd.read_csv(INFOGROUP_2022[0], nrows=10000, dtype=str)
df = pd.read_csv(INFOGROUP_2022[0], dtype=str)

In [4]:
# covert all the elements of these columns to strings
df['SIC CODE'] = df['SIC CODE'].astype(str)
df['SIC CODE 1'] = df['SIC CODE 1'].astype(str)
df['SIC CODE 2'] = df['SIC CODE 2'].astype(str)
df['SIC CODE 3'] = df['SIC CODE 3'].astype(str)
df['SIC CODE 4'] = df['SIC CODE 4'].astype(str)
            

In [8]:
# create new empty dataframe (to append to) containing only SIC Codes that we 0291
df_sic_0291 = pd.DataFrame(columns=df.columns)

for i in range(len(df)):
    if ( (df.iloc[i]['SIC CODE'].__contains__('0291') ) |
        (df.iloc[i]['SIC CODE 1'].__contains__('0291')) |
        (df.iloc[i]['SIC CODE 2'].__contains__('0291')) |
        (df.iloc[i]['SIC CODE 3'].__contains__('0291')) |
        (df.iloc[i]['SIC CODE 4'].__contains__('0291'))
        ):
        # df_sic_0291.append(df.iloc[i])
        df_sic_0291.loc[len(df_sic_0291.index)] = df.iloc[i]

In [9]:
df_sic_0291.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,...,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,CHURCH RANCH,20009 HIGHWAY 72,ARVADA,CO,80007,8236,059,303,2,A,...,5,060500,3,039.869196,-105.230716,P,19740,2,216,08059
1,UNITED PRODUCERS INC,3491 E STATE ROAD 44,RUSHVILLE,IN,46173,7846,139,765,2,B,...,6,974400,3,039.615453,-085.379248,P,00000,NaN,000,18139
2,IVEY RANCH COUNTRY CLUB HOA,74580 VARNER RD,THOUSAND PALMS,CA,92276,4201,065,760,2,C,...,5,044520,1,033.789523,-116.359597,P,40140,2,348,06065
3,DALE SYBRANT RANCH,45240 859TH RD,BASSETT,NE,68714,6077,149,402,2,A,...,3,974600,1,042.282034,-099.453784,P,00000,NaN,000,31149
4,BRANDT CATTLE CO,299 W MAIN ST,BRAWLEY,CA,92227,2240,025,760,2,D,...,7,010500,4,032.978750,-115.546533,0,20940,2,000,06025


In [11]:
FILENAME = "../data/code_0291.csv"
df_sic_0291.to_csv(FILENAME)

In [5]:
# create new empty dataframe (to append to) containing only SIC Codes: 0241
df_sic_0241 = pd.DataFrame(columns=df.columns)


for i in range(len(df)):
# for i in range(100000):
    if ( ( df.iloc[i]['SIC CODE'].__contains__('0241') ) |
         ( df.iloc[i]['SIC CODE 1'].__contains__('0241') ) |
         ( df.iloc[i]['SIC CODE 2'].__contains__('0241') ) |
         ( df.iloc[i]['SIC CODE 3'].__contains__('0241') ) |
         ( df.iloc[i]['SIC CODE 4'].__contains__('0241') )
        ):
        # df_sic_0241.append(df.iloc[i])
        df_sic_0241.loc[len(df_sic_0241.index)] = df.iloc[i]

In [6]:
df_sic_0241.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,...,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,DAIRYNET INC,2301 RESEARCH PARK WAY # 155,BROOKINGS,SD,57006,1724,011,605,2,B,...,7,958900,5,044.320347,-096.764656,P,15100,1,000,46011
1,PRAIRIE FARMS DAIRY INC,808 E MAIN ST,STREATOR,IL,61364,3004,099,815,2,C,...,7,964300,2,041.121418,-088.825205,P,36837,1,176,17099
2,K C DAIRY LLP,48612 212TH ST,ELKTON,SD,57026,6716,011,605,2,D,...,3,958600,2,044.314913,-096.471455,P,15100,1,000,46011
3,R & D DAIRY INC,52825 HIGHWAY 275,OAKDALE,NE,68761,3013,003,402,2,A,...,2,979800,3,042.072087,-097.945441,P,00000,NaN,000,31003
4,STOP & GO,5566 ALEXIS RD,SYLVANIA,OH,43560,2456,095,419,2,A,...,7,008100,1,041.716807,-083.677325,P,45780,2,534,39095


In [7]:
FILENAME = "../data/code_0241.csv"
df_sic_0241.to_csv(FILENAME)